In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
from keras.utils import to_categorical
import random
from tensorflow import set_random_seed
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

[nltk_data] Downloading package wordnet to C:\Users\sreekar
[nltk_data]     chidurala\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


In [2]:
train1=pd.read_csv("train.tsv",sep="\t")

In [3]:
test1=pd.read_csv("test.tsv",sep="\t")

In [4]:
stop_words=nltk.corpus.stopwords.words('english')
stop_words=list(np.setdiff1d(stop_words,['not']))
junk_words=[]
len(stop_words)
stop_words.extend(junk_words)
len(stop_words)

178

In [5]:
docs=train1['Phrase'].fillna('').str.lower().str.replace('[^a-z ]','')
docs.head()

0    a series of escapades demonstrating the adage ...
1    a series of escapades demonstrating the adage ...
2                                             a series
3                                                    a
4                                               series
Name: Phrase, dtype: object

In [7]:
stemmer=nltk.PorterStemmer()
def clean_text(row_text):
    #print(type(row_text))
    row_words=row_text.split(' ')
    #print(row_words)
    row_words= [stemmer.stem(word) for word in row_words if word not in stop_words]
    #print(row_words)
    #print('----')
    return ' '.join(row_words)

docs_clean=docs.apply(lambda v: clean_text(v))

In [8]:
train,test=train_test_split(docs_clean,test_size=.3,random_state=100)
train_y=train1.loc[train.index]['Sentiment']
test_y=train1.loc[test.index]['Sentiment']

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
cv.fit(train)
train_x=cv.transform(train)
test_x=cv.transform(test)

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [12]:
adaboost_model=RandomForestClassifier(n_estimators=3,random_state=100)
adaboost_model.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=False, random_state=100,
                       verbose=0, warm_start=False)

In [13]:
pred_class=adaboost_model.predict(test_x)


In [14]:
from sklearn.metrics import accuracy_score,f1_score,roc_curve,auc

In [15]:
print(accuracy_score(test_y,pred_class))

0.6197189115297536


In [ ]:
#training with whole data

In [16]:
test_docs=test1['Phrase'].fillna('').str.lower().str.replace('[^a-z ]','')
test_docs.head()

0    an intermittently pleasing but mostly routine ...
1    an intermittently pleasing but mostly routine ...
2                                                   an
3    intermittently pleasing but mostly routine effort
4           intermittently pleasing but mostly routine
Name: Phrase, dtype: object

In [17]:
test_clean=test_docs.apply(lambda v: clean_text(v))

In [19]:
train1.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
cv.fit(docs_clean)
train_x=cv.transform(docs_clean)
test_x=cv.transform(test_clean)

In [23]:
train_y=train1["Sentiment"]

In [24]:
adaboost_model=RandomForestClassifier(n_estimators=7,random_state=100)
adaboost_model.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=7,
                       n_jobs=None, oob_score=False, random_state=100,
                       verbose=0, warm_start=False)

In [26]:
pred=adaboost_model.predict(test_x)

In [27]:
sub_file = pd.read_csv('sampleSubmission.csv',sep=',')
sub_file.Sentiment=pred
sub_file.to_csv('Submission_ensemble.csv',index=False)

In [28]:
sub_file.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
